In [1]:
%matplotlib inline
import os, sys, pickle
import numpy as np
import pandas as pd

In [2]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000


In [3]:
!ls -a

.				    .ipynb_checkpoints
..				    judge_embeddings
100CASELEVEL_Touse.dta		    process_data.ipynb
auburn_appct_stata.dta		    results
auburn_district_w_songer_codes.dta  results.zip
case_by_judge.csv		    testing-and-plotting-vectors.ipynb
case_by_judge_imp_feats.csv	    test-similarity.ipynb
caselevel_touse.dta		    Untitled.ipynb
docvec_index_id.txt		    V3CircuitJudgesBIO.dta
docvecs_0.txt			    V3DistrictJudgesBIO.dta
docvecs_1.txt			    vectors2.log
docvecs_2.txt			    vectors2.py
docvecs.p			    vectors3.py
get-judge-name-id-map.ipynb	    vectors.log
get-sim-scores.ipynb		    vectors.py
id_case_map.csv


In [15]:
final_judges = pd.read_csv('final_737_judges_all_feats.csv')

In [16]:
final_judges.shape

(737, 36)

In [17]:
final_judges.head()

,caseid,songername,jOrigname,judgefirstname,judgemiddlename,judgelastname,judgeidentificationnumber,Author,AppointmentDate1,Circuitjudge1,birthyear,State,placeofbirthstate,presidentname,genderNew,raceorethnicity,partyaffiliationofpresident,nameofschool1,degree1,nameofschool2,degree2,party,StateOfResidence,SchoolState1,Public1,Private1,SchoolState2,Public2,Private2,x_ageon40orless,x_ageon40s,x_ageon50s,x_ageon60s,x_ageon70ormore,startdate,experience
0,X101L14003,"MOTZ, DIANA GRIBBON",MOTZ,DIANA,JANE GRIBBON,MOTZ,1705,MOTZ,1994-06-16,4,1943,MD,DC,William J. Clinton,F,White,Democratic,Vassar College,B.A.,University of Virginia School of Law,J.D.,1,20,NY,0,1,VA,1,0,0,0,1,0,0,1994-06-16,10
1,X10284I003,"WILKINSON, J. HARVIE, III",WILKINSON,JAMES,HARVIE,WILKINSON,2587,WILKINSON,1984-08-13,4,1944,VA,NY,Ronald Reagan,M,White,Republican,Yale University,B.A.,University of Virginia School of Law,J.D.,0,46,CT,0,1,VA,1,0,0,1,0,0,0,1984-08-13,20
2,X102UPO003,"SLOVITER, DOLORES K.",SLOVITER,DOLORES,KORMAN,SLOVITER,2208,RENDELL,1979-06-21,3,1932,PA,PA,Jimmy Carter,F,White,Democratic,Temple University,A.B.,University of Pennsylvania Law School,LL.B.,1,38,PA,1,0,PA,0,1,0,1,0,0,0,1979-06-21,25
3,X103660N,"HUG, PROCTER R., JR.",HUG,PROCTER,RALPH,HUG,1112,NaN,1977-09-15,9,1931,NV,NV,Jimmy Carter,M,White,Democratic,"University of Nevada, Reno",B.S.,Stanford Law School,LL.B.,1,28,NV,1,0,CA,0,1,0,1,0,0,0,1977-09-15,17
4,X1036DGN,"SCIRICA, ANTHONY J.",SCIRICA,ANTHONY,JOSEPH,SCIRICA,2128,NaN,1984-09-18,3,1940,PA,PA,Ronald Reagan,M,White,Republican,Wesleyan University,B.A.,University of Michigan Law School,J.D.,0,38,CT,0,1,MI,1,0,0,1,0,0,0,1987-08-06,14


In [38]:
def read_judge_sim_file(fname):
    with open('judge_similarities_top20.csv','r') as f:
        temp = f.readlines()
        
    f = map(lambda x: x.rstrip("\n").split(","),temp)
    
    new_f = []
    for x in f:
        new_f.append([int(x[0]),int(x[1]),float(x[2])])
    return new_f

def get_index_from_judgeid(ids):
    id1 = ids[0]
    id2 = ids[1]
    ind1 = final_judges[final_judges.judgeidentificationnumber==id1].index[0]
    ind2 = final_judges[final_judges.judgeidentificationnumber==id2].index[0]
    return [ind1,ind2]

In [30]:
new_f = read_judge_sim_file('judge_similarities_top20.csv')

In [31]:
new_f

[[1180, 2374, 0.2956268191],
 [390, 1314, 0.2968933284],
 [729, 176, 0.2956267893],
 [1221, 1510, 0.2900688648],
 [997, 3233, 0.290068835],
 [3272, 2063, 0.2894240618],
 [3015, 1776, 0.2876123488],
 [155, 2876, 0.2894240618],
 [1840, 2834, 0.2824738622],
 [1927, 2892, 0.2876123488],
 [1326, 247, 0.197219193],
 [883, 2653, 0.2792617083],
 [2661, 16, 0.2792616785],
 [2186, 170, 0.2781262994],
 [35, 466, 0.2755073905],
 [125, 675, 0.2781262994],
 [739, 1119, 0.2697028816],
 [689, 1948, 0.2755073905],
 [678, 443, 0.269656986],
 [1206, 1401, 0.2697028816]]

In [32]:
top5 = map(lambda x: [x[0],x[1]], new_f[:5])

In [39]:
final_judges.ix[get_index_from_judgeid(top5[0]),:]

,caseid,songername,jOrigname,judgefirstname,judgemiddlename,judgelastname,judgeidentificationnumber,Author,AppointmentDate1,Circuitjudge1,birthyear,State,placeofbirthstate,presidentname,genderNew,raceorethnicity,partyaffiliationofpresident,nameofschool1,degree1,nameofschool2,degree2,party,StateOfResidence,SchoolState1,Public1,Private1,SchoolState2,Public2,Private2,x_ageon40orless,x_ageon40s,x_ageon50s,x_ageon60s,x_ageon70ormore,startdate,experience
534,X2K43L,"JOHNSON, FRANK M.",JOHNSON,FRANK,MINIS,JOHNSON,1180,RONEY,1956-02-01,11,1918,AL,AL,Jimmy Carter,M,White,Reassignment,University of Alabama School of Law,LL.B.,NaN,NaN,0,NaN,AL,1,0,NaN,NaN,NaN,0,0,0,1,0,1981-10-01,8
184,X12F2V0003,"THOMPSON, DAVID R.",THOMPSON,DAVID,R.,THOMPSON,2374,THOMPSON,1985-12-17,9,1930,CA,CA,Ronald Reagan,M,White,Republican,University of Southern California,"B.S., business",University of Southern California Law School,LL.B.,0,5,CA,0,1,CA,0,1,0,0,1,0,0,1985-12-17,20


In [40]:
final_judges.ix[get_index_from_judgeid(top5[1]),:]

,caseid,songername,jOrigname,judgefirstname,judgemiddlename,judgelastname,judgeidentificationnumber,Author,AppointmentDate1,Circuitjudge1,birthyear,State,placeofbirthstate,presidentname,genderNew,raceorethnicity,partyaffiliationofpresident,nameofschool1,degree1,nameofschool2,degree2,party,StateOfResidence,SchoolState1,Public1,Private1,SchoolState2,Public2,Private2,x_ageon40orless,x_ageon40s,x_ageon50s,x_ageon60s,x_ageon70ormore,startdate,experience
576,X2NNHS,"CARTER, JAMES M.",CARTER,JAMES,MARSHALL,CARTER,390,HUFSTEDLER,1949-10-18,9,1904,CA,CA,Lyndon B. Johnson,M,White,Democratic,Pomona College,A.B.,Harvard Law School,,1,5,CA,0,1,MA,0,1,0,0,0,1,0,1967-11-16,11
41,X1059SU003,"KOZINSKI, ALEX",KOZINSKI,ALEX,,KOZINSKI,1314,SILVERMAN,1985-11-07,9,1950,DC,Romania,Ronald Reagan,M,White,Republican,"University of California, Los Angeles",A.B.,"University of California, Los Angeles, School ...",J.D.,0,5,CA,1,0,CA,1,0,1,0,0,0,0,1985-11-07,19


In [41]:
final_judges.ix[get_index_from_judgeid(top5[2]),:]

,caseid,songername,jOrigname,judgefirstname,judgemiddlename,judgelastname,judgeidentificationnumber,Author,AppointmentDate1,Circuitjudge1,birthyear,State,placeofbirthstate,presidentname,genderNew,raceorethnicity,partyaffiliationofpresident,nameofschool1,degree1,nameofschool2,degree2,party,StateOfResidence,SchoolState1,Public1,Private1,SchoolState2,Public2,Private2,x_ageon40orless,x_ageon40s,x_ageon50s,x_ageon60s,x_ageon70ormore,startdate,experience
499,X2AICU,"FAGG, GEORGE B.",FAGG,GEORGE,GARDNER,FAGG,729,FAGG,1982-10-01,8,1934,IA,IA,Ronald Reagan,M,White,Republican,Drake University,B.A.,Drake University Law School,J.D.,0,15,IA,0,1,IA,1,0,0,1,0,0,0,1982-10-01,15
85,X1176EQ003,"BIRCH, STANLEY F., JR.",BIRCH,STANLEY,F.,BIRCH,176,PER CURIAM,1990-05-14,11,1945,GA,VA,George H.W. Bush,M,White,Republican,University of Virginia,B.A.,Emory University School of Law,J.D.,0,10,VA,1,0,GA,0,1,0,1,0,0,0,1990-05-14,14


In [42]:
final_judges.ix[get_index_from_judgeid(top5[3]),:]

,caseid,songername,jOrigname,judgefirstname,judgemiddlename,judgelastname,judgeidentificationnumber,Author,AppointmentDate1,Circuitjudge1,birthyear,State,placeofbirthstate,presidentname,genderNew,raceorethnicity,partyaffiliationofpresident,nameofschool1,degree1,nameofschool2,degree2,party,StateOfResidence,SchoolState1,Public1,Private1,SchoolState2,Public2,Private2,x_ageon40orless,x_ageon40s,x_ageon50s,x_ageon60s,x_ageon70ormore,startdate,experience
428,X21KH4,"KALODNER, HARRY E.",KALODNER,HARRY,ELLIS,KALODNER,1221,STALEY,1939-05-04,3,1896,PA,PA,Harry S Truman,M,White,Democratic,University of Pennsylvania Law School,LL.B.,NaN,NaN,1,38,PA,0,1,NaN,NaN,NaN,0,0,1,0,0,1946-07-27,16
53,X10FA8K003,"MAYER, HALDANE ROBERT",MAYER,HALDANE,ROBERT,MAYER,1510,MAYER,1987-06-15,0,1941,DC,NY,Ronald Reagan,M,White,Republican,U.S. Military Academy,B.S.,William and Mary Law School,J.D.,0,NaN,NY,1,0,VA,1,0,0,1,0,0,0,1987-06-15,17


In [43]:
final_judges.ix[get_index_from_judgeid(top5[4]),:]

,caseid,songername,jOrigname,judgefirstname,judgemiddlename,judgelastname,judgeidentificationnumber,Author,AppointmentDate1,Circuitjudge1,birthyear,State,placeofbirthstate,presidentname,genderNew,raceorethnicity,partyaffiliationofpresident,nameofschool1,degree1,nameofschool2,degree2,party,StateOfResidence,SchoolState1,Public1,Private1,SchoolState2,Public2,Private2,x_ageon40orless,x_ageon40s,x_ageon50s,x_ageon60s,x_ageon70ormore,startdate,experience
447,X21KLQ,"HASTINGS, JOHN",HASTINGS,JOHN,SIMPSON,HASTINGS,997,SCHNACKENBERG,1957-08-26,7,1898,IN,IN,Dwight D. Eisenhower,M,White,Republican,U.S. Military Academy,B.S.,Indiana University School of Law -- Bloomingto...,LL.B.,0,14,NY,1,0,IN,1,0,0,0,1,0,0,1957-08-26,5
376,X1E45F8003,"THOMPSON, OJETTA",THOMPSON,OJETTA,ROGERIEE,THOMPSON,3233,DYK,2010-03-30,1,1951,RI,SC,Barack Obama,F,African American,Democratic,Brown University,A.B.,Boston University School of Law,J.D.,1,NaN,RI,0,1,MA,0,1,0,0,1,0,0,2010-03-30,0
